<a href="https://colab.research.google.com/github/TariqAlhathloul/Smart-Street/blob/main/WorkSpace/train_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## linkes for the project

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow.svg)](https://universe.roboflow.com/capstoneteam/road-lines-segmentation-3)

[![GitHub Repoistory](https://badges.aleen42.com/src/github.svg)](https://github.com/TariqAlhathloul/Smart-Street)


## installation

In [1]:
!pip install ultralytics -q
!pip install roboflow -q
!pip install ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.8/882.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.1 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd
import os

from google.colab import userdata
API_KEY = userdata.get('ROBOFLOW_API_KEY')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-phoc6")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Road-lines-segmentation-2 in yolov11:: 100%|██████████| 52208/52208 [00:08<00:00, 6035.26it/s]


## upload the dataset

In [4]:
TRAIN_DIR_IAMAGES = '/content/Road-lines-segmentation-2/train/images'
VAL_DIR_IAMGES = '/content/Road-lines-segmentation-2/valid/images'
TEST_DIR_IMAGES = '/content/Road-lines-segmentation-2/test/images'

In [5]:
print(f"Total Training: {len(os.listdir(TRAIN_DIR_IAMAGES))} images")
print(f"Total Validation: {len(os.listdir(VAL_DIR_IAMGES))} images")
print(f"Total Testing: {len(os.listdir(TEST_DIR_IMAGES))} images")

Total Training: 24198 images
Total Validation: 1400 images
Total Testing: 500 images


## check if the GPU is avalabie

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct  6 14:23:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## upload the model

In [8]:
model = YOLO("yolo11n-seg.pt")

100%|██████████| 5.90M/5.90M [00:00<00:00, 21.7MB/s]


In [9]:
model.info()

YOLO11n-seg summary: 355 layers, 2,876,848 parameters, 0 gradients, 10.5 GFLOPs


(355, 2876848, 0, 10.5294976)

## overwrite the data.yaml file

In [10]:
%%writefile /content/Road-lines-segmentation-2/data.yaml
train: /content/Road-lines-segmentation-2/train/images
val: /content/Road-lines-segmentation-2/valid/images
test: /content/Road-lines-segmentation-2/test/images

nc: 3
names: ['License_Plate', 'car', 'solid-yellow-line']

roboflow:
  workspace: capstoneteam
  project: road-lines-segmentation-phoc6
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-phoc6/dataset/2


Overwriting /content/Road-lines-segmentation-2/data.yaml


## Hyperparameters, finetuning

In [11]:
IMG_SIZE = 640
BATCH_SIZE = -1
EPOCHS = 100

In [12]:
results = model.train(
    data = '/content/Road-lines-segmentation-2/data.yaml',
    task = 'segment',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Ex-11',
    project = 'Ex-11',
    patience = 20,
    save = True,
    optimizer = 'SGD',
    lr0 = 0.001,
    lrf = 0.0001,
    momentum = 0.937,
    weight_decay = 0.005,
    cos_lr=True,
    profile=True,
    freeze=10,
    pretrained=True,
    dropout=0.2
)

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/content/Road-lines-segmentation-2/data.yaml, epochs=100, time=None, patience=20, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Ex-11, name=Ex-11, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=93, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=True, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

100%|██████████| 755k/755k [00:00<00:00, 4.28MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 20.9MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.10G reserved, 0.06G allocated, 14.59G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2843193       10.36         0.254         42.84           nan        (1, 3, 640, 640)                    list
     2843193       20.72         0.403         29.79           nan        (2, 3, 640, 640)                    list
     2843193       41.43         0.742         29.19           nan        (4, 3, 640, 640)                    list
     2843193       82.86         1.447         38.63           nan        (8, 3, 640, 640)                    list
     2843193       165.7         2.772         55.75           nan       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 51 for CUDA:0 8.85G/14.75G (60%) ✅


...

    100/100      5.91G     0.6006     0.8596     0.3935     0.8438         86        640: 100%|██████████| 475/475 [04:15<00:00,  1.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:12<00:00,  1.11it/s]

                   all       1400       3892      0.908      0.862      0.909      0.735      0.906      0.856      0.904      0.625



100 epochs completed in 7.576 hours.
Optimizer stripped from Ex-11/Ex-11/weights/last.pt, 6.0MB
Optimizer stripped from Ex-11/Ex-11/weights/best.pt, 6.0MB

Validating Ex-11/Ex-11/weights/best.pt...
Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-seg summary (fused): 265 layers, 2,835,153 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:13<00:00,  1.01it/s]


                   all       1400       3892      0.904      0.865      0.911      0.736      0.904      0.858      0.904      0.627
         License_Plate        231        255      0.911      0.745      0.823      0.512      0.903       0.73       0.81      0.494
                   car       1087       2432      0.875      0.889       0.94      0.808      0.876      0.883      0.936      0.714
     solid-yellow-line       1161       1205      0.926      0.959      0.968      0.888      0.932       0.96      0.967      0.673
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to Ex-11/Ex-11


## evaluation on the validationset


In [13]:
evaluation = model.val(
    data = '/content/Road-lines-segmentation-2/data.yaml',
    task = 'segment',
    imgsz = 640,
    iou = 0.6,
    batch = 32,
    save_json = True,
    save_txt = True,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-evaluation-11'
)

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-seg summary (fused): 265 layers, 2,835,153 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning /content/Road-lines-segmentation-2/valid/labels.cache... 1400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1400/1400 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/Road-lines-segmentation-2/valid/images/front-left-NO20240903-145041-000844F_MP4-0244_jpg.rf.4202a8af1a35ceb1e66ab38357ef22b8.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [01:09<00:00,  1.57s/it]


                   all       1400       3892      0.909      0.865      0.912      0.738      0.906      0.862      0.909      0.594
         License_Plate        231        255      0.919      0.745      0.828      0.514      0.906      0.733      0.817      0.494
                   car       1087       2432      0.874      0.891      0.941      0.812      0.876      0.891      0.942       0.72
     solid-yellow-line       1161       1205      0.933      0.959      0.967      0.888      0.936      0.962      0.967      0.568
Speed: 0.3ms preprocess, 5.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Saving Ex-11/Model-segment-evaluation-11/predictions.json...
Results saved to Ex-11/Model-segment-evaluation-11


In [ ]:
"""
Previous results wiht YOLOv8n-seg
Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
val: Scanning /content/Prototype-4/valid/labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:08<00:00,  8.23s/it]
                   all          7         18      0.816      0.592      0.803      0.681      0.816      0.592      0.828      0.623
                   car          7          9          1      0.296      0.755       0.61          1      0.296      0.805      0.644
     solid-yellow-line          7          9      0.632      0.889      0.851      0.752      0.632      0.889      0.851      0.602
Speed: 1.4ms preprocess, 14.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Saving Model-prototype/Model-segment-evaluation-52/predictions.json...
Results saved to Model-prototype/Model-segment-evaluation-52
"""

## predictions on the testset

In [ ]:
predictions = model.predict(
    data = '/content/Road-lines-segmentation-2/data.yaml',
    source = '/content/Road-lines-segmentation-2/test/images',
    task = 'segment',
    imgsz = 640,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-predictions',
    save_json = True,
)


image 1/3 /content/Prototype-4/test/images/IMG-20241003-WA0008_jpg.rf.8659e8c7fc1ab4e86064fe16e1c1e130.jpg: 640x640 1 solid-yellow-line, 15.2ms
image 2/3 /content/Prototype-4/test/images/IMG-20241003-WA0014_jpg.rf.eba2af291897f7af6fa5f89b4c43632e.jpg: 640x640 (no detections), 14.8ms
image 3/3 /content/Prototype-4/test/images/IMG-20241003-WA0024_jpg.rf.3e44c3b05f5ab3e1735d4d65c0c5ca64.jpg: 640x640 (no detections), 14.9ms
Speed: 1.6ms preprocess, 15.0ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


## apply quantization

In [14]:
model.export(format="onnx", int8=True, dynamic=True,half=True, simplify=True, data='/content/Road-lines-segmentation-2/data.yaml')

Ultralytics 8.3.5 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
WARNING ⚠️ half=True and int8=True are mutually exclusive, setting half=False.

PyTorch: starting from 'Ex-11/Ex-11/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 39, 8400), (1, 32, 160, 160)) (5.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim==0.1.34', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 295.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 287.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 245.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 269.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 13.4s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim==0.1.34', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime

'Ex-11/Ex-11/weights/best.onnx'

In [16]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [17]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## download results

In [18]:
!zip -r model-11.zip /content/Ex-11/Ex-11

  adding: content/Ex-11/Ex-11/ (stored 0%)
  adding: content/Ex-11/Ex-11/labels_correlogram.jpg (deflated 33%)
  adding: content/Ex-11/Ex-11/val_batch0_labels.jpg (deflated 5%)
  adding: content/Ex-11/Ex-11/train_batch42750.jpg (deflated 6%)
  adding: content/Ex-11/Ex-11/val_batch2_pred.jpg (deflated 6%)
  adding: content/Ex-11/Ex-11/train_batch1.jpg (deflated 1%)
  adding: content/Ex-11/Ex-11/train_batch42752.jpg (deflated 4%)
  adding: content/Ex-11/Ex-11/MaskPR_curve.png (deflated 13%)
  adding: content/Ex-11/Ex-11/args.yaml (deflated 51%)
  adding: content/Ex-11/Ex-11/BoxF1_curve.png (deflated 10%)
  adding: content/Ex-11/Ex-11/labels.jpg (deflated 40%)
  adding: content/Ex-11/Ex-11/val_batch2_labels.jpg (deflated 6%)
  adding: content/Ex-11/Ex-11/val_batch1_labels.jpg (deflated 6%)
  adding: content/Ex-11/Ex-11/train_batch42751.jpg (deflated 6%)
  adding: content/Ex-11/Ex-11/train_batch2.jpg (deflated 1%)
  adding: content/Ex-11/Ex-11/MaskP_curve.png (deflated 14%)
  adding: conte

In [20]:
!zip -r evaluation-11.zip /content/Ex-11/Model-segment-evaluation-11

  adding: content/Ex-11/Model-segment-evaluation-11/ (stored 0%)
  adding: content/Ex-11/Model-segment-evaluation-11/val_batch0_labels.jpg (deflated 5%)
  adding: content/Ex-11/Model-segment-evaluation-11/val_batch2_pred.jpg (deflated 6%)
  adding: content/Ex-11/Model-segment-evaluation-11/predictions.json (deflated 82%)
  adding: content/Ex-11/Model-segment-evaluation-11/MaskPR_curve.png (deflated 13%)
  adding: content/Ex-11/Model-segment-evaluation-11/labels/ (stored 0%)
  adding: content/Ex-11/Model-segment-evaluation-11/labels/front-left-NO20240908-150138-000147F_MP4-0038_jpg.rf.76a1fd5375871da316fb297a01e59bb7.txt (deflated 72%)
  adding: content/Ex-11/Model-segment-evaluation-11/labels/front-right-NO20240903-143941-000833F_MP4-0062_jpg.rf.dbbc098da30ecb071b39c6b33d0dd6dd.txt (deflated 80%)
  adding: content/Ex-11/Model-segment-evaluation-11/labels/front-right-NO20240909-181340-000011F_MP4-0011_jpg.rf.792a3543a92dbec9ec0fb408e807fcd6.txt (deflated 84%)
  adding: content/Ex-11/Mod

## deploy the model on roboflow

In [21]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-phoc6")
project.version(2).deploy(model_type="yolov11n", model_path="/content/Ex-11/Ex-11")

loading Roboflow workspace...
loading Roboflow project...
An error occured when getting the model upload URL: This version already has a trained model. Please generate and train a new version in order to upload model to Roboflow.
